🛠️ What is a Tool in LangChain?
🔹 A Tool is a callable function or utility that the LLM can invoke to perform tasks that are outside its own knowledge or capabilities.
Think of it like an external plugin or helper function that gives the LLM superpowers, such as:

Searching Wikipedia

Fetching academic papers

Retrieving private documents (using a vectorstore)

Calling an API


🧠 Analogy
🧠 LLM = Human brain
🧰 Tool = Calculator, Library, Phone, Web browser

If a human is answering a question and doesn't know the answer, they:

Ask someone (Wikipedia tool)

Look it up in their notes (retriever tool)

Search Google Scholar (Arxiv tool)

The same applies here. LLMs can "use tools" like humans do.

⚙️ What Does a Tool Contain?
In LangChain, a Tool is a Python object that includes:


| Component          | Description                                                            |
| ------------------ | ---------------------------------------------------------------------- |
| `name`             | A short string name used by the LLM to call the tool                   |
| `description`      | Tells the LLM **when to use** this tool                                |
| `func` or `invoke` | The actual Python function to execute when the tool is used            |
| `args_schema`      | (Optional) A schema for validating and auto-generating input arguments |



🔽 Under the Hood: How a Tool Works
Let's say you define this tool:


from langchain.tools import Tool

def search_docs(query: str) -> str:
    return "Some search results for: " + query

my_tool = Tool(
    name="doc_search",
    description="Useful for answering questions about internal docs.",
    func=search_docs
)

LangChain under the hood wraps this into a JSON-like function call schema


✅ So the LLM sees:
"When the user asks about internal documentation, I can call a function doc_search(query) with a string input."




🔁 Flow of Tool Usage
Here’s what happens step-by-step in a tool call:

🔹 Step 1: User Input
User asks something like:

Tell me about LangSmith’s observability features.

🔹 Step 2: LLM Analyzes Input
The LLM realizes it needs external help, based on the tool descriptions.


🔹 Step 3: Tool Call Generation
LLM generates a structured call like this:

{
  "tool_call": {
    "name": "langsmith-search",
    "arguments": {
      "query": "LangSmith’s observability features"
    }
  }
}


🔹 Step 4: LangChain Executes the Tool
LangChain takes the tool name and args

It calls the associated Python function:
retriever.invoke("LangSmith’s observability features")


🔹 Step 5: Result is Returned to LLM
Tool returns a string like:
LangSmith provides real-time tracing, error analysis, and observability dashboards...

🔹 Step 6: LLM Composes Final Answer
LLM reads the result and generates a human-friendly answer:

"LangSmith offers rich observability including real-time tracing and dashboards that help developers understand LLM behavior..."



      +------------------------+
      |   User Asks Question   |
      | "What is LangSmith?"   |
      +-----------+------------+
                  |
                  v
        +---------+----------+
        |      LLM (Gemma)   |
        |  + Prompt & Tools  |
        +---------+----------+
                  |
          Decides tool is needed
                  |
                  v
   +--------------+-------------------+
   |     Tool Call by LLM             |
   | name = "langsmith-search"        |
   | args = { "query": "LangSmith" }  |
   +--------------+-------------------+
                  |
                  v
      +-----------+-----------+
      |  LangChain Tool Runs  |
      |  retriever.invoke()   |
      +-----------+-----------+
                  |
       Gets relevant documents
                  |
                  v
       +----------+----------+
       |  Tool returns result |
       | "LangSmith helps..." |
       +----------+----------+
                  |
        LLM uses result to respond
                  |
                  v
     +------------+------------+
     |  Final Answer to User   |
     |  "LangSmith provides..."|
     +-------------------------+


In [2]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)

d:\LANGCHAIN_PROJECTS\Q&A_Chat_bot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020303685820>, search_kwargs={})

In [1]:
# creating a custom tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith")
retriever_tool.name

NameError: name 'retriever' is not defined

✅ What it does:
Takes your FAISS retriever (built using LangSmith docs) and wraps it as a tool.

The name is what the LLM will refer to when calling the tool.

The description is used by the LLM to decide when to use this tool during a conversation.

📌 Now the LLM knows:

"If the user is asking about LangSmith, I should consider using the langsmith-search tool."

In [8]:
retriever_tool

Tool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000203050C72E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020303685820>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000203050C76A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020303685820>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), do

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key = groq_api_key,model_name="gemma2-9b-it")

In [11]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

📜 Load a Tool Calling Prompt
✅ What it does:
Pulls a pre-defined chat prompt from the LangChain Hub.

"hwchase17/openai-functions-agent" is a standard function-calling agent prompt.

It includes:

System message describing tool usage.

Placeholders like {tools} and {tool_names}.

Instructions to guide the LLM to use tools when necessary.

In [13]:
## Arxiv--Research
## Tools creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [14]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [15]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


🧱 The Two-Layer Design
When using tools like Wikipedia or Arxiv in LangChain, you are working with a two-layer architecture:
| Layer                 | Class                                     | Role                                                      |
| --------------------- | ----------------------------------------- | --------------------------------------------------------- |
| **1. Low-level**      | `WikipediaAPIWrapper` / `ArxivAPIWrapper` | Wraps the raw API logic (fetching, filtering, formatting) |
| **2. Tool Interface** | `WikipediaQueryRun` / `ArxivQueryRun`     | Converts the wrapper into a usable LangChain `Tool`       |

1️⃣ WikipediaAPIWrapper / ArxivAPIWrapper
✅ Purpose:
This is the low-level utility that:

Knows how to query a source (Wikipedia or Arxiv)

Parses and cleans up the data

Limits or filters the content (e.g., top-k, max chars)

✅ What it does:
Formats your query for Wikipedia or Arxiv

Sends the request (usually via requests or httpx)

Parses the result (e.g., HTML → plain text)

Trims it based on parameters like:

top_k_results: How many results to return

doc_content_chars_max: Max characters in content

✅ Why it's called a "Wrapper":
Because it wraps the actual API — hiding the raw HTTP calls and providing a clean, reusable Python function.

wiki_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_wrapper.run("LangChain")
# → Returns a string summary of LangChain from Wikipedia


2️⃣ WikipediaQueryRun / ArxivQueryRun
✅ Purpose:
This wraps the wrapper into a LangChain Tool interface so it can be used by agents.

Think of it as the Tool adapter that turns your wrapper into a fully-functioning LangChain Tool.

✅ What it does:
Takes the WikipediaAPIWrapper or ArxivAPIWrapper

Exposes a .invoke() method compatible with LangChain Tools

Auto-generates input/output schema (function calling)

Adds a default name, description, and callable

wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki_tool.invoke("LangChain")
# Internally calls: wiki_wrapper.run("LangChain")
# Returns the Wikipedia content in a Tool-compatible way


💡 Why Not Just Use the Wrapper?
Good question!

You can use the wrapper directly in your own Python code, but for LangChain Agents, tools must follow a specific interface:

Have a name

Have a description

Be callable via .invoke() with a single input

Return a string (or ToolOutput)

That's exactly what QueryRun classes are for — they standardize the wrapper into a Tool.




🧠 Analogy
Think of Wrapper as a "Car Engine"
And QueryRun as the "Steering Wheel and Dashboard"

You need both to actually drive the car (use the tool in a LangChain agent).


In [18]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name

'langsmith-search'

1. 📥 Input: retriever
This is any object that implements BaseRetriever, like:

Chroma.as_retriever()

This retriever must implement the method:

retriever.invoke(query: str) -> List[Document]

Which returns the top-k relevant documents for a query.


2. 🧠 Output: a Tool object
The Tool object looks like this under the hood:

Tool(
    name="vector_search",
    description="Search for relevant information...",
    func=retriever_function,
    coroutine=None,  # optional for async
    args_schema=None # auto-generated from retriever
)
So now the retriever is callable as a tool, with name/description that the LLM can use to call it properly in agent settings.

3. ⚙️ Internal Logic
Here’s what create_retriever_tool actually does:

  -->Wraps the retriever in a Tool with a callable function retriever.invoke.
 
  -->Auto-generates schema so that the agent knows the input (usually just a    query: str).

  -->Provides a name and description so the LLM knows when to use this tool.

This lets you use it like:
from langchain.agents import initialize_agent

agent = initialize_agent(
    tools=[retriever_tool],
    llm=ChatGroq(),
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

🧠 Bonus: Behind the Scenes
Here's a simplified version of how LangChain defines it:

def create_retriever_tool(retriever, name: str, description: str) -> Tool:
    def retriever_func(query: str) -> str:
        docs = retriever.invoke(query)
        return "\n\n".join([doc.page_content for doc in docs])
    
    return Tool(
        name=name,
        description=description,
        func=retriever_func
    )

So essentially:

It takes a query → runs it through the retriever → returns the combined document content as a string.

In [19]:
tools=[wiki,arxiv,retriever_tool]

✅ What it does:
Collects all three tools:

Wikipedia (general purpose)

Arxiv (research papers)

LangSmith FAISS Retriever (custom docs)

This list will be passed into the agent so it knows what tools it can use.

In [20]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

✅ What it does:
Creates an LLM-powered agent using OpenAI-style tool calling schema.

Behind the scenes, it sets up:

The ChatPromptTemplate with instructions and tool names.

Logic for interpreting LLM tool calls and routing them.

Wrapping the tools into function-callable specs.

Now the agent can:

Read your prompt.

Decide to call one of the tools.

Combine the results into a coherent answer.

This uses the OpenAI-compatible function-calling spec, which Groq’s model can handle.

In [21]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

✅ What it does:
Wraps the agent into an executor, which handles:

Repeated calls

Input formatting

Tracing (if enabled)

Tool usage display (verbose mode)

verbose=True prints step-by-step what the agent is doing, including:

What tool it picks

What query it sends

What result it gets

Final response from the LLM

In [22]:
agent_executor.invoke({"input":"Tell me about Langsmith"})




> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. \n'}

In [23]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without being explicitly programmed.

> Finished chain.


{'input': 'What is machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without being explicitly programmed.'}

so how many times an llm call is invoked as we can see that llm has to invoke to decide a tool then that tool give some output this output is sent to the llm and then the llm takes that output and invokes once again and give the final output is it like that



🔁 Number of LLM Calls in a Tool-Based Agent Flow
✅ In a typical AgentExecutor (like in LangChain ReAct or OpenAI function agents), the LLM is called multiple times per user question:
🔄 Step-by-Step Breakdown (ReAct or Function-Calling Agent):
🧩 Step 1: First LLM Call — Decide on Action
LLM is called with the user query and tool descriptions.

It returns something like this:
Thought: I need to look up information about LangSmith.
Action: langsmith-search
Action Input: LangSmith observability features
✔️ LLM Call #1 happens here to decide which tool to use.

🔧 Step 2: LangChain Executes Tool
LangChain reads:

Action = langsmith-search

Action Input = LangSmith observability features

So it calls:
retriever.invoke("LangSmith observability features")
This gives some text output (e.g., retrieved docs or API result).

🧠 No LLM call here — only the tool function runs.

🧠 Step 3: Second LLM Call — Final Answer
LangChain then calls the LLM again, passing:

The original question

The tool output (observation)

And asks:

"Now, using this observation, answer the user's question."
Observation: LangSmith provides tracing features to monitor...
Thought: I now understand how to answer.
Final Answer: LangSmith’s observability helps developers...

[User Question] 
      |
      v
[LLM Call #1 → Decides tool]
      |
      v
[LangChain runs tool]
      |
      v
[LLM Call #2 → Observes & decides next step]
      |
      ├── (Done?) → [Final Answer] ✅
      └── (Need another tool?) → [Repeat]




✅ Two Main Ways to Use LangChain Agents
1. initialize_agent(...)

search_agent = initialize_agent(...)
response = search_agent.run(input)

2. AgentExecutor(...)
agent_executor = AgentExecutor(agent=..., tools=..., verbose=True)
response = agent_executor.invoke({"input": "..."})


initialize_agent(...) already returns a fully wrapped AgentExecutor by default.


initialize_agent(...) ⇒ returns an instance of AgentExecutor
